[1] First, install facebook-sdk

# pip install facebook-sdk

[2] log into your Facebook account 

[3] go to https://developers.facebook.com/tools/explorer
    
[4] copy your access token       

In [ ]:
# Copy and paste in the value you just got from the inline frame into this variable and execute this cell.
# Keep in mind that you could have just gone to https://developers.facebook.com/tools/explorer/
# and "Get User Access Token" 
access_token = ''

In [ ]:
import facebook
import json
import requests

Social media data are in json format containing curly braces so it is helpful to use a function to print json data with indent. 

In [ ]:
# function to print the output pretty ("pretty print")
def pp(input):
    print json.dumps(input, indent=1)

# Me

In [ ]:
graph = facebook.GraphAPI(access_token)
me = graph.get_object('me')
print me

In [ ]:
pp(me)

In [ ]:
# get your posts
posts = graph.get_connections('me','posts')
pp(posts)

In [ ]:
for post in posts['data']:
    print pp(post)

In [ ]:
for post in posts['data']:
    print post['message'], post['created_time']

In [ ]:
#save your posts in txt file
f = open("input.txt","w")
f.write(json.dumps(posts,indent=4))
f.close()
print len(posts['data'])

I have only a few posts on my facebook page.

If you have many posts, you won't see all of them in one page. Your program needs to traverse all the pages to get all the posts. See the example below 

In [ ]:
graph = facebook.GraphAPI(access_token)
posts = graph.get_connections('me','posts')

# this process could take a ver long if you have a large volume of posts
allmyposts = []

# Wrap this block in a while loop so we can keep paginating requests until finished.
while(True):
    try:
        for post in posts['data']:
            allmyposts.append([post['message'], post['created_time']])
        # Attempt to make a request to the next page of data, if it exists.
        posts=requests.get(posts['paging']['next']).json()
    except KeyError:
        # When there are no more pages (['paging']['next']), break from the loop and end the script.
        break
        
print allmyposts

In [ ]:
len(allmyposts)

In [ ]:
# to save all the data

f = open("input.txt","w")

graph = facebook.GraphAPI(access_token)
posts = graph.get_connections('me','posts')

# this process could take a ver long if you have a large volume of posts
allmyposts = []

# Wrap this block in a while loop so we can keep paginating requests until finished.
while(True):
    try:
        #for post in posts['data']:
        f.write(json.dumps(posts,indent=4))
        # Attempt to make a request to the next page of data, if it exists.
        posts=requests.get(posts['paging']['next']).json()
    except KeyError:
        # When there are no more pages (['paging']['next']), break from the loop and end the script.
        break

#close file
f.close()

# Your friends

In [ ]:
pp(graph.get_connections('me', 'friends'))

In [ ]:
friends = graph.get_connections('me', 'friends')
for i in friends['data']:
    print i['name']

This is my FB page for teaching Facebook Ananalytics so it has only a few friends (former MBA students). What if you have many friends? Your program needs to traverse all the data until pagin ends. (http://stackoverflow.com/questions/28589239/python-facebook-api-cursor-pagination)

In [ ]:
allfriends = []

# Wrap this block in a while loop so we can keep paginating requests until
# finished.
while(True):
    try:
        for friend in friends['data']:
            allfriends.append(friend['name'].encode('utf-8'))
        # Attempt to make a request to the next page of data, if it exists.
        friends=requests.get(friends['paging']['next']).json()
    except KeyError:
        # When there are no more pages (['paging']['next']), break from the
        # loop and end the script.
        break
print allfriends

# Search by Keywords

In [ ]:
mysearch = graph.request("search", {'q' : 'Kansas State University', 'type' : 'page', 'limit' : 5})
mysearch

In [ ]:
for i in mysearch['data']:
    print i

In [ ]:
for i in mysearch['data']:
    print i['name'], i['category']

# Results for a Graph API query for CNN

+ More examples http://facebook-sdk.readthedocs.io/en/latest/api.html

In [ ]:
# Get an instance of CNN.com
# Using the page name also works if you know it.
# e.g. 'CNN' or 'CNN News'
cnn_id = '10154246192721509'
cnn = pp(graph.get_object(cnn_id))
cnn

In [ ]:
cnn_page = graph.get_connections(id='10154246192721509', connection_name='likes')
cnn_page

# Querying the Graph API for Open Graph objects by their URLs

In [ ]:
# CNN article "All eyes on Trump, Ryan relationship after health care defeat"
pp(graph.get_object('http://www.cnn.com/2017/03/26/politics/ryan-trump-relationship-health-care/index.html?sr=fbCNN032717ryan-trump-relationship-health-care0130AMVODtopLink&linkId=35871059'))


# Comparing likes between Coke and Pepsi fan pages

In [ ]:
# Find Pepsi and Coke in search results

pp(graph.request('search', {'q' : 'pepsi', 'type' : 'page', 'limit' : 5}))
pp(graph.request('search', {'q' : 'coke', 'type' : 'page', 'limit' : 5}))

# Use the ids to query for likes

pepsi_id = '56381779049' # Could also use 'PepsiUS'
coke_id = '40796308305'  # Could also use 'CocaCola'

# A quick way to format integers with commas every 3 digits
def int_format(n): return "{:,}".format(n)

print "Pepsi likes:", int_format(graph.get_object(pepsi_id)['likes'])
print "Coke likes:", int_format(graph.get_object(coke_id)['likes'])

# Querying for all of your friends' likes

In [ ]:
base_url = 'https://graph.facebook.com/me'

# Get 10 likes for 10 friends
fields = 'id,name,friends.limit(10).fields(likes.limit(10))'

url = '%s?fields=%s&access_token=%s' % \
    (base_url, fields, access_token,)

content = requests.get(url).json()

content

In [ ]:
for i in content['friends']['data']:
    for j in i['likes']['data']:
        print j

In [ ]:
for i in content['friends']['data']:
    for j in i['likes']['data']:
        print j['name'], j['category'], j['created_time']

# Calculating the most popular likes among your friends

In [ ]:
friends = graph.get_connections("me", "friends")['data']

likes = { friend['name'] : graph.get_connections(friend['id'], "likes")['data'] 
          for friend in friends }

print likes

In [ ]:
# Analyze all likes from friendships for frequency

# pip install prettytable
from prettytable import PrettyTable
from collections import Counter
friends_likes = Counter([like['name']
                         for friend in likes 
                           for like in likes[friend]
                               if like.get('name')])

pt = PrettyTable(field_names=['Name', 'Freq'])
pt.align['Name'], pt.align['Freq'] = 'l', 'r'
[ pt.add_row(fl) for fl in friends_likes.most_common(10) ]

print 'Top 10 likes amongst friends'
print pt

# Calculating the most popular categories for likes among your friends

In [ ]:
# Analyze all like categories by frequency

friends_likes_categories = Counter([like['category'] 
                                    for friend in likes 
                                      for like in likes[friend]])

pt = PrettyTable(field_names=['Category', 'Freq'])
pt.align['Category'], pt.align['Freq'] = 'l', 'r'
[ pt.add_row(flc) for flc in friends_likes_categories.most_common(10) ]

print "Top 10 like categories for friends"
print pt

# Calculating the number of likes for each friend and sorting by frequency

In [ ]:
# Build a frequency distribution of number of likes by 
# friend with a dictionary comprehension and sort it in 
# descending order

from operator import itemgetter

num_likes_by_friend = { friend : len(likes[friend]) 
                        for friend in likes }


pt = PrettyTable(field_names=['Friend', 'Num Likes'])
pt.align['Friend'], pt.align['Num Likes'] = 'l', 'r'
[ pt.add_row(nlbf) 
  for nlbf in sorted(num_likes_by_friend.items(), 
                     key=itemgetter(1), 
                     reverse=True) ]

print "Number of likes per friend"
print pt

# Finding common likes between an ego and its friendships in a social network

In [ ]:
# Which of your likes are in common with which friends?
my_likes = [ like['name'] 
             for like in graph.get_connections("me", "likes")['data'] ]

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[ pt.add_row((ml,)) for ml in my_likes ]
print "My likes"
print pt

# Use the set intersection as represented by the ampersand
# operator to find common likes.

common_likes = list(set(my_likes) & set(friends_likes))

pt = PrettyTable(field_names=["Name"])
pt.align = 'l'
[ pt.add_row((cl,)) for cl in common_likes ]
print
print "My common likes with friends"
print pt

# Reference

- https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/ipynb/Chapter%202%20-%20Mining%20Facebook.ipynb
- http://shivammitra.com/pagination-facebook-api-python/

# Additional Resources

- Chapter 4.  Posts, Pages, and User Interactions on Facebook in Mastering Social Media Mining with Python (As a student of K-State, you can access over hundreds of data analytics books at no charge through Safari Books Online (http://apps.lib.k-state.edu/databases/?s=safari).
- Netvizz (a facebook app for collecting network data)
    - https://apps.facebook.com/netvizz/
- Facepager (a program for collecting posts and comments) 
    - Installation https://github.com/strohne/Facepager/releases
    - Tutorial https://www.youtube.com/watch?v=S9kYApoR8U4